Long Short-Term Memory network is a type of Recurrent Neural Network

In [38]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [39]:
# fix random seed for reproducibility
numpy.random.seed(7)


In [40]:
# load the dataset
dataframe = pandas.read_csv('csv/2108.csv', usecols=[0], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
print len(dataset)

99


In [41]:
dataframe.describe()

,4
count,99.000000
mean,21.070707
std,18.158740
min,2.000000
25%,8.000000
50%,14.000000
75%,29.500000
max,107.000000


In [165]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
print len(dataset)

99


In [166]:
# split into train and test sets - train set ends on May 12
train_size = 50
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size-10:len(dataset),:]
print(len(train), len(test))

(50, 59)


In [167]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [168]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [169]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [170]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=100)

Epoch 1/100
48/48 [==============================] - 0s - loss: 0.0243     
Epoch 2/100
48/48 [==============================] - 0s - loss: 0.0236     
Epoch 3/100
48/48 [==============================] - 0s - loss: 0.0229     
Epoch 4/100
48/48 [==============================] - 0s - loss: 0.0221     
Epoch 5/100
48/48 [==============================] - 0s - loss: 0.0216     
Epoch 6/100
48/48 [==============================] - 0s - loss: 0.0209     
Epoch 7/100
48/48 [==============================] - 0s - loss: 0.0203     
Epoch 8/100
48/48 [==============================] - 0s - loss: 0.0197     
Epoch 9/100
48/48 [==============================] - 0s - loss: 0.0192     
Epoch 10/100
48/48 [==============================] - 0s - loss: 0.0187     
Epoch 11/100
48/48 [==============================] - 0s - loss: 0.0182     
Epoch 12/100
48/48 [==============================] - 0s - loss: 0.0177     
Epoch 13/100
48/48 [==============================] - 0s - loss: 0.0173     
Epoch 14

In [171]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.6f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.6f RMSE' % (testScore))

Train Score: 0.001035 RMSE
Test Score: 0.002294 RMSE


In [172]:
testPredict

array([[ 0.09817643],
       [ 0.09845764],
       [ 0.09848638],
       [ 0.09848638],
       [ 0.09826144],
       [ 0.09833257],
       [ 0.0967675 ],
       [ 0.09783681],
       [ 0.09838998],
       [ 0.09838998],
       [ 0.09807742],
       [ 0.09833257],
       [ 0.08438438],
       [ 0.09807742],
       [ 0.09629372],
       [ 0.09653798],
       [ 0.0967675 ],
       [ 0.09753869],
       [ 0.09796424],
       [ 0.09807742],
       [ 0.09826144],
       [ 0.09769499],
       [ 0.09134869],
       [ 0.09516954],
       [ 0.09838998],
       [ 0.09826144],
       [ 0.09796424],
       [ 0.09783681],
       [ 0.0967675 ],
       [ 0.09343648],
       [ 0.07249352],
       [ 0.0365277 ],
       [ 0.07478135],
       [ 0.07982156],
       [ 0.09343648],
       [ 0.09603471],
       [ 0.09807742],
       [ 0.09304687],
       [ 0.07699513],
       [ 0.09516954],
       [ 0.0967675 ],
       [ 0.09381188],
       [ 0.07982156],
       [ 0.08438438],
       [ 0.0954726 ],
       [ 0

In [173]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict-10)+(look_back)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

ValueError: could not broadcast input array from shape (57,1) into shape (48,1)

lookback = 1, hidden neurons = 4
Train Score: 398.36 RMSE
Test Score: 314.96 RMSE

lookback = 3, hidden neurons = 4
Train Score: 387.69 RMSE
Test Score: 214.87 RMSE

lookback = 7, hidden neurons = 4
Train Score: 357.99 RMSE
Test Score: 146.85 RMSE

lookback = 10, hidden neurons = 8
Train Score: 366.99 RMSE
Test Score: 149.24 RMSE

